In [59]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from http import HTTPStatus
import json 
import dashscope
from openai import AsyncOpenAI
from typing import List, Dict, Tuple, Any
import re

In [60]:
load_dotenv()

True

In [61]:
XAI_API_KEY = os.getenv("XAI_API_KEY", "")
XAI_BASE_URL = os.getenv("XAI_BASE_URL", "")
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "")
QDRANT_URL = os.getenv("QDRANT_URL", "")

In [62]:
client = AsyncOpenAI(
    base_url=XAI_BASE_URL,
    api_key=XAI_API_KEY
)

In [63]:
def get_current_temperature(location: str, unit: str = "fahrenheit"):
    temperature: int
    if unit.lower() == "fahrenheit":
        temperature = 59
    elif unit.lower() == "celsius":
        temperature = 15
    else:
        raise ValueError("unit must be one of fahrenheit or celsius")
    return {"location": location, "temperature": temperature, "unit": "fahrenheit"}


def get_current_ceiling(location: str):
    return {
        "location": location,
        "ceiling": 15000,
        "ceiling_type": "broken",
        "unit": "ft",
    }

tools_map = {
    "get_current_temperature": get_current_temperature,
    "get_current_ceiling": get_current_ceiling,
}


In [64]:
qdrant_client = QdrantClient(url=QDRANT_URL)


dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

def embed_with_str(query: str):
    resp = dashscope.TextEmbedding.call(
        model=dashscope.TextEmbedding.Models.text_embedding_v3,
        api_key=DASHSCOPE_API_KEY,
        input=query)
    if resp.status_code == HTTPStatus.OK:
        return resp.output['embeddings'][0]['embedding']
    else:
        print(resp)

def search_knowledge_base(query: str) -> Tuple[List[Dict[str, Any]], str]:
    """
    Search the knowledge base for relevant information.

    Args:
        query: str -> query used to retrieve the relevant information.
    Returns:
        Tuple containing:
        1. List of the formatted results with title, content preview, and source link
        2. String joining all relevant information from results
    """

    embed = embed_with_str(query)
    results = qdrant_client.query_points(
        collection_name="knowledge_base_collection",
        query=embed,
        with_payload=True,
        limit=3,
        score_threshold=0.6
    )
    formatted_results = []
    combined_text_parts = []
    for result in results.points:
        # Format dictionary result
        content_preview = result.payload["content"][:200] + "..." if len(result.payload["content"]) > 200 else result.payload["content"]
        formatted_results.append({
            "title": result.payload["title"],
            "content_preview": content_preview,
            "source_link": result.payload["source_link"],
            "relevance_score": round(result.score, 3)
        })

        # Add to combined text
        combined_text_parts.append(
            f"Title: {result.payload['title']}\n"
            f"Content: {result.payload['content']}\n"
            f"Source: {result.payload['source_link']}\n"
        )

    combined_text = "\n".join(combined_text_parts)
    return formatted_results, combined_text

def search_doctors(query: str) -> Tuple[List[Dict[str, Any]], str]:
    """
    Search for doctors based on query.
    Returns:
        Tuple containing:
        1. List of formatted results with doctor information
        2. String joining all relevant information from results
    """
    embed = embed_with_str(query)
    results = qdrant_client.query_points(
        collection_name="doctor_collection",
        query=embed,
        with_payload=True,
        limit=3,
        score_threshold=0.5,
    )

    formatted_results = []
    combined_text_parts = []

    for result in results.points:
        # Format dictionary result
        formatted_results.append({
            "doctor_name": result.payload["doctor_name"],
            "specialization": result.payload["doctor_field"],
            "description": result.payload["doctor_description"],
            "availability_status": "Available" if result.payload["availability"] else "Not Available",
            "appointment_link": result.payload["appointment_link"],
            "relevance_score": round(result.score, 3)
        })

        # Add to combined text
        combined_text_parts.append(
            f"Specialization: {result.payload['doctor_field']}\n"
            f"Description: {result.payload['doctor_description']}\n"
        )

    combined_text = "\n".join(combined_text_parts)
    return formatted_results, combined_text

In [65]:
tools_definition = [
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Retrieve health information from the knowledge base given the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user query, e.g. what is diabetes ?"
                    },
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_doctors",
            "description": "Get the suitalbe doctors based on the user's query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user query, e.g. recommend me a doctor for diabetes"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

tools_map = {
    "search_knowledge_base": search_knowledge_base,
    "search_doctors": search_doctors,
}


In [83]:
global messages
messages = []

In [84]:
messages = [{"role": "user", "content": "Hi, what is acne ?"}]

In [85]:

async def get_next_questions(last_message: str):
    """Based on the last message get suggestions for follow up questions."""
    try:
        prompt = {
            "role": "user",
            "content": f"""Based on this health-related response: "{last_message}"
            Suggest 2 natural follow-up questions that a patient might want to ask.
            These should be within 4 four words.
            Return ONLY the questions in a Python list format like this: ["question 1", "question 2"]
            The questions should be clear and concise."""
        }
        response = await client.chat.completions.create(
            model="grok-2-1212",
            messages=[prompt],
            temperature=0.7
        )

        suggestions_text = response.choices[0].message.content
        import ast
        try:
            suggestions = ast.literal_eval(suggestions_text)
            if isinstance(suggestions, list) and len(suggestions) > 0:
                return suggestions[:2]  # Ensure we only get 2 questions
        except:
            pass

        return ["Could you explain more about that?",
                "Summarize this"]
    except Exception as e:
        print(f"Error generating suggestions: {e}")
        return ["Could you explain more about that?",
                "Summarize this"]

In [86]:
async def routerLLM(tools_definition, user_msg, assistant_msg=None):
    try:
        if assistant_msg:
            prompt = f"""
            Tools definition: {tools_definition}.\n

            Based on the previous assistant message : {assistant_msg} and 
            the current user query : {user_msg} determine if any of the tools
            in the tools definition should be used or not. 

            Rules for tool selection: 
            1. The 'search_knowledge_base' tool:
                - Use when health related information needs to be retrieved to answer the user query. 
                - The user query should be a new query and not a follow up to the previous assistant answer.

            2. The 'search_doctors' tool:
                - Use when the user wants to consult or asking for a doctor recommendation.  
            Provide your response in the following JSON format:
                {{
                    "requires_tool": true/false,
                    "tool_name": "name of tool if required, null if not",
                    "reason": "detailed explanation of why the tool is needed or not",
                    "args": {{
                        "query": The user query used to retrieve information from the necessary tool.
                    }}
                }}

            Important:
            - Only return a valid JSON object
            - Don't include the actual content/code in the response
            - Be decisive - either a tool is needed or it isn't
            - Consider the context from the previous assistant message
            """
        else:
            prompt = f"""
            Tools definition: {tools_definition}.\n

            Based on the user query : {user_msg} determine if any of the tools
            in the tools definition should be used or not. 

            Rules for tool selection: 
            1. The 'search_knowledge_base' tool:
                - Use when health related information needs to be retrieved to answer the user query. 
                - If the user query is not health related, DO NOT use this tool.

            2. The 'search_doctors' tool:
                - Use when the user wants to consult or asking for a doctor recommendation.  

            Provide your response in the following JSON format:
                {{
                    "requires_tool": true/false,
                    "tool_name": "name of tool if required, null if not",
                    "reason": "detailed explanation of why the tool is needed or not",
                    "args": {{
                        "query": The user query used to retrieve information from the necessary tool.
                    }}
                }}
        
            Important:
            - Only return a valid JSON object
            - Don't include the actual content/code in the response
            - Be decisive - either a tool is needed or it isn't
            """

        response = await client.chat.completions.create(
            model="grok-2-1212",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        result = response.choices[0].message.content
        json_content = result.strip().strip('```json').strip('```')
        print(json_content)
        json_result = json.loads(json_content)
        return json_result
    except Exception as e:
        pass

In [87]:
result = await routerLLM(tools_definition, "doctor for acne", assistant_msg="Acne is a form of inflammatory skin on the face.")
print(result)


{
    "requires_tool": true,
    "tool_name": "search_doctors",
    "reason": "The user's query 'doctor for acne' indicates a need for a doctor recommendation specifically for acne, which aligns with the purpose of the 'search_doctors' tool. This query is a direct request for a recommendation and not a follow-up to the previous assistant message about the definition of acne.",
    "args": {
        "query": "doctor for acne"
    }
}

{'requires_tool': True, 'tool_name': 'search_doctors', 'reason': "The user's query 'doctor for acne' indicates a need for a doctor recommendation specifically for acne, which aligns with the purpose of the 'search_doctors' tool. This query is a direct request for a recommendation and not a follow-up to the previous assistant message about the definition of acne.", 'args': {'query': 'doctor for acne'}}


In [100]:
if messages[-1]['role'] == 'user':
    usr_msg = messages[-1]['content']
    if len(messages) > 1 and messages[-2]['role'] == 'assistant':
        assistant_msg = messages[-2]['content']
    else:
        assistant_msg = None

    print('user message', usr_msg)
    print('assistant message', assistant_msg)
    route = await routerLLM(tools_definition, usr_msg, assistant_msg)

    if route['requires_tool']:
        fn_name = route['tool_name']
        fn_args = route['args']
        result, _ = tools_map[fn_name](**fn_args)
        print(result)

        # prompt = f"""
        #     User query: {usr_msg}
        #     Retrieved data: {result}
        # """
        messages.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_name": fn_name
            }
        )
    response = await client.chat.completions.create(
        model="grok-2-1212",
        messages=messages,
        stream=True
    )

    fn_call_in_progress = False
    fn_results = []

    async for chunk in response:

        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)


        if chunk.choices[0].delta.tool_calls:
            print("\n")
            
            for tool_call in chunk.choices[0].delta.tool_calls:

                fn_name = tool_call.function.name
                fn_args = json.loads(tool_call.function.arguments)


                result = tools_map[fn_name](**fn_args)

                messages.append(
                    {
                        "role": "tool",
                        "content": json.loads(result),
                        "tool_name": fn_name,
                        "tool_call_id": tool_call.id  # tool_call.id supplied in Grok's response
                    }
                )

if messages[-1]['role'] == 'tool':
    response = await client.chat.completions.create(
        model="grok-2-1212",
        messages=messages,
        stream=True
    )

    messages.append(
        {
            "role": "assistant",
            "content": ""
        }
    )

    async for chunk in response:
        if chunk.choices[0].delta.content is not None:
            messages[-1]['content'] += chunk.choices[0].delta.content 
            print(chunk.choices[0].delta.content, end="", flush=True)  

if messages[-2]['role'] == 'tool' and messages[-1]['role'] == 'assistant':
    print("\n")
    suggestions = await get_next_questions(messages[-1]['content'])
    print('suggestions', suggestions)
    

user message recommend me a doctor for eye sight
assistant message Based on the available information, I recommend Dr. Azlina Firzah, who specializes in Endocrinology and is currently available for appointments. Endocrinologists often deal with metabolic issues, including those related to weight management. You can book an appointment with her through this link: [Appointment with Dr. Azlina Firzah](https://www.pantai.com.my/kuala-lumpur/ms/appointment/azlina-firzah-bt-abd-aziz).

If you are specifically looking for a doctor focused on weight loss and Dr. Vijay Ananda Paramasvaran becomes available, he might be a good choice as well, given his specialization in endocrinology, which often includes managing conditions related to weight. Keep an eye on his availability status at this link: [Appointment with Dr. Vijay Ananda Paramasvaran](https://www.pantai.com.my/kuala-lumpur/appointment/vijay-ananda-paramasvaran).

{
    "requires_tool": true,
    "tool_name": "search_doctors",
    "reaso

In [99]:
messages.append({
    'role': 'user',
    'content': 'recommend me a doctor for eye sight'
})